## Training neural networks

This notebook describes the workflow for training neural networks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import getcwd
from pathlib import Path
from functools import reduce

import yaml
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from dstools.reproducibility import make_filename
from dstools.reproducibility.util import git_hash_in_path, get_version
from dstools.util import save

from yass import util

Using TensorFlow backend.


In [3]:
here = getcwd()
here_version = git_hash_in_path(here)

In [4]:
# for reference
print('YASS version is: {}'.format(get_version('yass')))
print('nbs version is: {}'.format(here_version))

YASS version is: d5925c9 options for specifying distribution of clean spikes
nbs version is: 1a9c9bb noteboks for new triage network


In [5]:
path_to_data = Path('~', 'data', 'triage').expanduser()
path_to_output = path_to_data  / 'models'
path_to_sets = path_to_data / 'sets'

## Loading training data

In [8]:
x_name = '2018-08-21T15-31-09:x-train-31wf7ch.npy'

path_to_x_train = path_to_sets / x_name
path_to_y_train = path_to_sets / x_name.replace('x', 'y')

path_to_x_test = path_to_sets / x_name.replace('train', 'test')
path_to_y_test = path_to_sets / x_name.replace('x-train', 'y-test')

x_train = np.load(path_to_x_train)
y_train = np.load(path_to_y_train)
x_test = np.load(path_to_x_test)
y_test = np.load(path_to_y_test)

In [9]:
def make_model(x_train, input_shape):
    n_data, window_size, n_channels, _ = x_train.shape

    model = Sequential()
        
#     model.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_last", padding='same'))
    
#     model.add(Dropout(0.75))

    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last", input_shape=input_shape))


#     model.add(Conv2D(70, kernel_size=(window_size, 1),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last"))


#     model.add(Conv2D(70, kernel_size=(1, n_channels),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='linear', use_bias=True,
                     data_format="channels_last"))
        
#     model.add(MaxPooling2D(pool_size=(1, 2), data_format="channels_last", padding='same'))
#     model.add(Dropout(0.75))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    opt = keras.optimizers.adam(lr=0.001)

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model


# Triage Ttaining

In [10]:
# import models
x_train = x_train[:, : , :, np.newaxis]
x_test = x_test[:, : , :, np.newaxis]

_, wf, ch, _ = x_train.shape

m = make_model(x_train, (wf, ch, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 31, 7, 10)         260       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 7, 10)         2510      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 7, 10)         2510      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2170)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2171      
Total params: 7,451
Trainable params: 7,451
Non-trainable params: 0
_________________________________________________________________


In [11]:
m.fit(x_train, y_train,
      batch_size=10000, epochs=100, shuffle=True,
      validation_data=(x_test, y_test))

Train on 48240 samples, validate on 23760 samples
Epoch 1/100
48240/48240 [==============================] - 17s 362us/step - loss: 0.6655 - acc: 0.6053 - val_loss: 0.5137 - val_acc: 0.8468
Epoch 2/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.4408 - acc: 0.8780 - val_loss: 0.3392 - val_acc: 0.9016
Epoch 3/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.2936 - acc: 0.9158 - val_loss: 0.2394 - val_acc: 0.9248
Epoch 4/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.2172 - acc: 0.9300 - val_loss: 0.1850 - val_acc: 0.9414
Epoch 5/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.1636 - acc: 0.9473 - val_loss: 0.1318 - val_acc: 0.9595
Epoch 6/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.1146 - acc: 0.9669 - val_loss: 0.0930 - val_acc: 0.9753
Epoch 7/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.0854 - acc: 0.9777 - val_loss: 0.0762 

48240/48240 [==============================] - 1s 28us/step - loss: 0.0172 - acc: 0.9959 - val_loss: 0.0177 - val_acc: 0.9964
Epoch 60/100
48240/48240 [==============================] - 1s 28us/step - loss: 0.0171 - acc: 0.9960 - val_loss: 0.0188 - val_acc: 0.9957
Epoch 61/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.0183 - acc: 0.9951 - val_loss: 0.0222 - val_acc: 0.9938
Epoch 62/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.0247 - acc: 0.9934 - val_loss: 0.0570 - val_acc: 0.9855
Epoch 63/100
48240/48240 [==============================] - 1s 27us/step - loss: 0.0427 - acc: 0.9885 - val_loss: 0.1559 - val_acc: 0.9640
Epoch 64/100
48240/48240 [==============================] - 1s 28us/step - loss: 0.1213 - acc: 0.9723 - val_loss: 0.3095 - val_acc: 0.9262
Epoch 65/100
48240/48240 [==============================] - 1s 28us/step - loss: 0.1680 - acc: 0.9615 - val_loss: 0.0370 - val_acc: 0.9895
Epoch 66/100
48240/48240 [==============

In [12]:
metadata = dict(path_to_x_train=str(path_to_x_train),
                path_to_y_train=str(path_to_y_train),
                path_to_x_test=str(path_to_x_test),
                path_to_y_test=str(path_to_y_test),
                yass_version=get_version('yass'),
                nb_version=git_hash_in_path(getcwd()))
metadata

{'path_to_x_train': '/home/Edu/data/triage/sets/2018-08-21T15-31-09:x-train-31wf7ch.npy',
 'path_to_y_train': '/home/Edu/data/triage/sets/2018-08-21T15-31-09:y-train-31wf7ch.npy',
 'path_to_x_test': '/home/Edu/data/triage/sets/2018-08-21T15-31-09:x-test-31wf7ch.npy',
 'path_to_y_test': '/home/Edu/data/triage/sets/2018-08-21T15-31-09:y-test-31wf7ch.npy',
 'yass_version': 'd5925c9 options for specifying distribution of clean spikes',
 'nb_version': '1a9c9bb noteboks for new triage network'}

In [13]:
_, wf, ch, _ = m.input_shape

sufix = f'triage-{wf}wf{ch}ch'
names = make_filename(sufix=sufix, extension=('h5', 'yaml'))

path_to_model, path_to_metadata = [str(path_to_output / name) for name in names]

m.save(path_to_model)
save(metadata, path_to_metadata)

In [14]:
print(path_to_model)

/home/Edu/data/triage/models/2018-08-21T15-35-13:triage-31wf7ch.h5
